In [ ]:
#import block
import tensorflow as tf
import h5py
import numpy as np
import json
import pandas as pd
import math
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
from google.colab import files

In [ ]:
#Function block for grabbing a section of the data
def grabData(hf, step=1):
  samples = hf['X'][::step]
  modulation_classes = hf['Y'][::step]
  

  mod_array = np.zeros(len(modulation_classes))
  i = 0
  for x in modulation_classes:
    mod_array[i] = np.where(x==1)[0][0]
    i = i + 1

  return samples, mod_array

In [ ]:
#MAIN BLOCK
def main():
  #if-else block for uploading hdf5 file
  user_input = input("Would you like to directly upload your hdf5(1) or import it from drive(2): ")
  if(int(user_input) == 1):
    hdf_upload = files.upload()
    hdf_val = list(hdf_upload.keys())
    hdf_key = hdf_val[0]
    
    user_input = input("Is the file uploaded zipped(1)? ")
    if(int(user_input) == 1):
      !unzip -qq $hdf_key
    
    hdf_name = hdf_key.split(".")
    hdf_name = hdf_name[0] + ".hdf5"
    
    #For simplification, hdf5 format should be a group for the samples(X) and a group for the label(Y)
    hdf5_file = h5py.File(hdf_name,  'r')
    print(list(hdf5_file.keys()))
  else:
    file_path = input("Enter in the path of the file in google drive")
    
    user_input = input("Is the file uploaded zipped(1)? ")
    if(int(user_input) == 1):
      !unzip -qq $file_path
      
      path_list = file_path.split("/")
      file_name = path_list[-1]
      file_split = file_name.split(".")
      file_raw = file_split[0]
      
      file_raw = file_raw + ".hdf5"
      hdf5_file = h5py.File(file_raw, 'r')
    
    hdf5_file = h5py.File(file_path, 'r')
    print(list(hdf5_file.keys()))

  #if-else block for uploading hdf5
  user_input = input("For the model, is it zipped(1) on google drive? ")
  if(int(user_input) == 1):
    file_path = input("Enter the file path for the model: ")
    !unzip -qq $file_path 
    file_split = file_path.split("/")
    file_split = file_split[-1].split(".")
    model_dir = file_split[0]
    model_dir = "content/" + model_dir
    print(model_dir)
    loaded_model = tf.keras.models.load_model(model_dir)
  else:
    file_path = input("Enter the file path for the model: ")
    loaded_model = tf.keras.models.load_model(file_path)
  
  #Grab data from the hdf5 file
  fraction = 1
  samples = hdf5_file['X'][::fraction]
  modulation = hdf5_file['Y'][::fraction]

  mod_array = np.zeros(len(modulation))
  i = 0
  for x in modulation:
    mod_array[i] = np.where(x==1)[0][0]
    i = i + 1

  #grab predictions, and check the accuracy percentage
  results = loaded_model.predict(samples)
  predictions = [p.argmax() for p in results]
  
  num_correct = 0
  for x in range(len(predictions)):
    print("Sample {0} | Model Prediction: {1} | Correct Label: {2}".format(x, predictions[x], mod_array[x]))
    if(mod_array[x] == predictions[x]):
      num_correct = num_correct + 1

  print("Total predictions made: {0}".format(len(predictions)))
  print("Precentage of correct predictions: {:.2%}".format(num_correct/len(predictions)))
    
  
  


if __name__ == "__main__":
    main()

In [ ]:
#SAVEDMODEL IMPORT BLOCK

#This script uses the SavedModel format for loading in a model
#The SavedModel format is a folder containing a complete tensorflow program
#containing complete model architecture along with trained parameters 

#Because of how the SavedModel format is stored, it's more convenient to store
#it google drive, and unzip directly from there to google colab

#use the following command to unzip the model to google colab
#! !unzip -qq '/path_to_SavedModel_in_google_drive/model.zip' 

#For testing
!unzip -qq '/content/drive/MyDrive/DS_shuffled.zip' 